In [2]:
import numpy as npy
import scipy.optimize as opt
import matplotlib.pyplot as plt

class currentProfile:
    def __init__(self, label):
        self.label=label
        self.current=[]

def dataReader(filename, label):
    inputfile=open(filename,'r')
    outputData=currentProfile(label)
    currentData=[]
    while(1):
        try:
            temp=next(inputfile).split()
            if(temp[4]==str(label)):
                outputData.current.append(1e6*float(temp[2]))
        except StopIteration:
            outputData.current=npy.array(outputData.current)
            break
    return outputData

def fitFunc(time, alpha, beta, omega):
    return 1+alpha*npy.cos(omega*time)+beta*npy.sin(omega*time)

In [3]:
length=73
angles=[2.5*i/180*npy.pi for i in range(length)]
measurements=[]
monitors=[]

for i in range(length):
    measurements.append(dataReader("data/measurement.txt",int(2*2.5*i)))
    monitors.append(dataReader("data/monitor.txt",int(2*2.5*i)))

In [4]:
ratios=[]
fitParameters=[]
fitParametersCOV=[]
for i in range(length):
    minLength=720
    ratios.append(measurements[i].current[:minLength]/monitors[i].current[:minLength])
    time=npy.array([0.5*j for j in range(minLength)])
    popt,pcov=opt.curve_fit(fitFunc, time, ratios[i], p0=[5,5,0.07])
    fitParameters.append(popt)
    fitParametersCOV.append(pcov)
    fitRatio=fitFunc(time,*popt)
    # fig,axs=plt.subplots(2,1)
    # axs[0].plot(time,ratios[i],label="Intensity")
    # axs[0].plot(time,fitRatio,label="Fitted")
    # axs[1].plot(time,(ratios[i]-fitRatio)/ratios[i],label="Residual")
    # axs[1].set_xlabel("Time (s)")
    # axs[0].set_ylabel("Intensity")
    # axs[1].set_ylabel("Residual")
    # axs[0].legend()
    # axs[1].legend()
    # fig.tight_layout()
    # fig.savefig("figure/RatioVsTime"+str(2.5*i)+"degree.jpg")
    # plt.close()

In [5]:
def alphaCurve(P, Ps, As, tanPsi, cosDelta, eta):
    alphaReal=(tanPsi**2 -npy.tan(P-Ps)**2) / (tanPsi**2 +npy.tan(P-Ps)**2)
    betaReal=(2 * tanPsi * cosDelta * npy.tan(P-Ps)) / (tanPsi**2 +npy.tan(P-Ps)**2)
    return (alphaReal*npy.cos(2*As)-betaReal*npy.sin(2*As))/eta

def betaCurve(P, Ps, As, tanPsi, cosDelta, eta):
    alphaReal=(tanPsi**2 -npy.tan(P-Ps)**2) / (tanPsi**2 +npy.tan(P-Ps)**2)
    betaReal=(2 * tanPsi * cosDelta * npy.tan(P-Ps)) / (tanPsi**2 +npy.tan(P-Ps)**2)
    return (alphaReal*npy.sin(2*As)+betaReal*npy.cos(2*As))/eta

def residualCurve(P, Ps, As, tanPsi, cosDelta, eta):
    return 1-alphaCurve(P, Ps, As, tanPsi, cosDelta, eta)**2-betaCurve(P, Ps, As, tanPsi, cosDelta, eta)**2

def refractionRate(tanPsi, cosDelta):
    rho=tanPsi*(cosDelta+1j*npy.sqrt(1-cosDelta**2))
    return npy.sin(65.0/180.0*npy.pi)**2+(npy.cos(65.0/180.0*npy.pi)**2)*((1-rho)/(1+rho))**2


In [6]:
alpha=[]
alphaError=[]
beta=[]
betaError=[]
residualError=[]
partLength=58
for i in range(length):
    alpha.append(fitParameters[i][0])
    beta.append(fitParameters[i][1])
    alphaError.append(npy.sqrt(fitParametersCOV[i][0,0]))
    betaError.append(npy.sqrt(fitParametersCOV[i][1,1]))
    residualError.append(2*npy.sqrt(fitParametersCOV[i][0,0]*(fitParameters[i][0])**2+fitParametersCOV[i][1,1]*(fitParameters[i][1])**2))

anglesFinal=npy.array(angles[:partLength])
alpha=npy.array(alpha[:partLength])
beta=npy.array(beta[:partLength])
alphaError=npy.array(alphaError[:partLength])
betaError=npy.array(betaError[:partLength])
residualError=npy.array(residualError[:partLength])
residual=1-alpha**2-beta**2

# (P, Ps, As, tanPsi, cosDelta, eta)
fig,axs=plt.subplots(3,1)
axs[0].errorbar(anglesFinal,alpha,alphaError,label="alpha")
popt1,pcov=opt.curve_fit(alphaCurve, anglesFinal, alpha, p0=[0.1,0.1,1,0.1,0.5], sigma=alphaError, maxfev=10000000, bounds=([0,0, -1, -1, 0],[npy.pi,npy.pi, 1,1, 10]))
axs[0].plot(anglesFinal,alphaCurve(anglesFinal, *popt1),label="Fit")

axs[1].errorbar(anglesFinal,beta,betaError,label="beta")
popt2,pcov=opt.curve_fit(betaCurve, anglesFinal, beta, p0=popt1, sigma=betaError, maxfev=10000000, bounds=([0,0, -1, -1, 0],[npy.pi,npy.pi, 1,1, 10]))
axs[1].plot(anglesFinal,betaCurve(anglesFinal, *popt2),label="Fit")

axs[2].errorbar(anglesFinal,residual,residualError,label="residual")
popt3,pcov=opt.curve_fit(residualCurve, anglesFinal, residual, p0=popt2, sigma=residualError, maxfev=10000000, bounds=([0,0, -1, -1, 0],[npy.pi,npy.pi, 1,1, 10]))
axs[2].plot(anglesFinal,residualCurve(anglesFinal, *popt3),label="Fit")

# axs[1].plot(angles,(ratios[i]-fitRatio)/ratios[i],label="Residual")
axs[2].set_xlabel("Angle")
axs[0].set_ylabel("alpha")
axs[1].set_ylabel("beta")
axs[2].set_ylabel("Residual")
axs[0].legend()
axs[1].legend()
axs[2].legend()
fig.tight_layout()
fig.savefig("figure/AlphaBeta.jpg")
plt.close()

print("(Ps, As, tanPsi, cosDelta, eta)"+str(popt1),"n="+str(refractionRate(popt1[2],popt1[3])))
print("(Ps, As, tanPsi, cosDelta, eta)"+str(popt2),"n="+str(refractionRate(popt2[2],popt2[3])))
print("(Ps, As, tanPsi, cosDelta, eta)"+str(popt3),"n="+str(refractionRate(popt3[2],popt3[3])))


(Ps, As, tanPsi, cosDelta, eta)[ 2.23898694  2.30641748  0.54321732 -0.93990985  0.15637178] n=(1.6766115873209886-1.2447036918407546j)
(Ps, As, tanPsi, cosDelta, eta)[ 2.20470021  2.35263826  0.59586065 -0.92378008  0.12236897] n=(1.3957050835401845-1.6274951279599188j)
(Ps, As, tanPsi, cosDelta, eta)[ 2.21423186  2.13718646  0.66674942 -0.76185644  0.12382333] n=(0.3959860258880249-0.9328675378212803j)
